In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

# Configuration
num_rows = 10_000
num_users = 250
start_date = pd.to_datetime("2024-09-02")
end_date = pd.to_datetime("2024-10-27")
date_range = pd.date_range(start=start_date, end=end_date)
verticals_categories= ['Restaurant', 'Turbo', 'CPGs', 'Travel']
rows_per_category = num_rows // len(verticals_categories)  # 2500

np.random.seed(50)  # For reproducibility

# Create weekly periods
weeks = pd.date_range(start=start_date, end=end_date, freq='W-MON')
num_weeks = len(weeks)

# STEP 1: Ensure every week has at least one first purchase
# Assign first purchase weeks to users (guarantee all weeks are covered)
user_first_purchase_week = {}
users_list = list(range(1, num_users + 1))
np.random.shuffle(users_list)

# First, assign one user to each week to guarantee coverage
for week_idx in range(num_weeks):
    if week_idx < len(users_list):
        user_id = users_list[week_idx]
        user_first_purchase_week[user_id] = week_idx

# Then assign remaining users to random weeks
remaining_users = users_list[num_weeks:] if num_weeks < len(users_list) else []
for user_id in remaining_users:
    user_first_purchase_week[user_id] = np.random.randint(0, num_weeks)

# STEP 2: For each user, create activity pattern starting from their first purchase week
user_active_weeks = {}
for user_id in range(1, num_users + 1):
    first_week = user_first_purchase_week[user_id]
    pattern = np.zeros(num_weeks, dtype=bool)
    
    # Always active in first purchase week
    pattern[first_week] = True
    
    # For subsequent weeks, create random activity pattern
    for week_idx in range(first_week + 1, num_weeks):
        # 60% chance of being active in any given week after first purchase
        pattern[week_idx] = np.random.choice([True, False], p=[0.6, 0.4])
    
    user_active_weeks[user_id] = pattern

# Generate columns
order_ids = np.arange(1, num_rows + 1)
user_ids = np.random.choice(np.arange(1, num_users + 1), size=num_rows)
verticals = np.array(verticals_categories * rows_per_category)
np.random.shuffle(verticals)

dates = []
user_order_count = {}

for i, user_id in enumerate(user_ids):
    if user_id not in user_order_count:
        user_order_count[user_id] = 0
    
    # Get user's active weeks pattern
    active_weeks = user_active_weeks[user_id]
    
    # Find which weeks this user is active
    active_week_indices = np.where(active_weeks)[0]
    
    if len(active_week_indices) > 0:
        # Choose a random active week for this order
        week_index = np.random.choice(active_week_indices)
        week_start = weeks[week_index]
        
        # Choose a random day within that week
        week_days = pd.date_range(start=week_start, periods=7, freq='D')
        # Only use days that are within our overall date range
        valid_days = [day for day in week_days if start_date <= day <= end_date]
        
        if valid_days:
            order_date = np.random.choice(valid_days)
            dates.append(order_date)
        else:
            # Fallback to random date
            dates.append(np.random.choice(date_range))
    else:
        # Fallback to random date if no active weeks
        dates.append(np.random.choice(date_range))
    
    user_order_count[user_id] += 1

# Create DataFrame
raw_data = pd.DataFrame({
    'ORDER_ID': order_ids,
    'DATE': dates,
    'USER_ID': user_ids,
    'VERTICAL': verticals
})

raw_data


,ORDER_ID,DATE,USER_ID,VERTICAL
0,1,2024-09-26,56,Travel
1,2,2024-10-21,244,Restaurant
2,3,2024-10-13,113,Restaurant
3,4,2024-09-27,126,Turbo
4,5,2024-10-19,187,Travel
...,...,...,...,...
9995,9996,2024-09-07,210,Turbo
9996,9997,2024-10-20,60,Turbo
9997,9998,2024-09-29,16,CPGs
9998,9999,2024-10-26,15,Turbo


In [16]:
raw_data['VERTICAL'].value_counts(normalize=True)


VERTICAL
Travel        0.25
Restaurant    0.25
Turbo         0.25
CPGs          0.25
Name: proportion, dtype: float64

In [17]:
raw_data['USER_ID'].nunique()

250

In [18]:
raw_data['ORDER_ID'].is_unique

True

In [19]:
#Export to CSV
raw_data.to_csv('../data/raw_data.csv', index=False)

In [20]:
from sqlalchemy import create_engine, Date, Integer, String
import os
from dotenv import load_dotenv

load_dotenv(dotenv_path='../.env')

user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
host = os.getenv("DB_HOST")
port = os.getenv("DB_PORT")
database = os.getenv("DB_NAME")

# Connect to PostgreSQL database
DATABASE_URL = f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}'
engine = create_engine(DATABASE_URL)


#Export to sql 
raw_data.to_sql(
    "RAW_DATA",
    engine,
    if_exists="replace",
    index=False,
    dtype={
        "ORDER_ID": Integer(),
        "DATE": Date(),
        "USER_ID": Integer(),
        "VERTICAL": String()
    }
)

print("Tabla subida con éxito.")

Tabla subida con éxito.
